In [1]:
import math
from tensorboardX import SummaryWriter

sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import env_configurations
import games_configurations
env_name = 'SuperMarioBros-v1'#'PongNoFrameskip-v4' #'MountainCar-v0' #'SuperMarioBros-v1'# 'PongNoFrameskip-v4' #SuperMarioBros-v1'
#env_name = 'CartPole-v1'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #

obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
config = games_configurations.mario_config_lstm

observation_shape = obs_space.shape
n_actions = action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

(None, 84, 84, 4)
6


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
from a2c_discrete import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

2019-07-05 15:08:53,742	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-05_15-08-53_2408/logs.
2019-07-05 15:08:53,858	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:22003 to respond...
2019-07-05 15:08:54,005	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:35123 to respond...
2019-07-05 15:08:54,012	INFO services.py:804 -- Starting Redis shard with 6.74 GB max memory.
2019-07-05 15:08:54,075	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-05_15-08-53_2408/logs.
2019-07-05 15:08:54,077	INFO services.py:1427 -- Starting the Plasma object store with 10.11 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:22003',
 'object_store_address': '/tmp/ray/session_2019-07-05_15-08-53_2408/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-05_15-08-53_2408/sockets/raylet',
 'webui_url': None}

In [5]:


agent = A2CAgent(sess,'pong', obs_space, True, action_space, config)
agent.train()


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
0
1
2
3
4
5
6
7
0
Instructions for updating:
Use tf.cast instead.
(pid=2487) 
(pid=2487) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=2487) For more information, please see:
(pid=2487)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=2487)   * https://github.com/tensorflow/addons
(pid=2487) If you depend on functionality not listed there, please file an issue.
(pid=2487) 
(pid=2494) 
(pid=2494) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=2494) For more information, please see:
(pid=2494)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset

Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/worker.py", line 1547, in print_logs
    msg = pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/rl/lib/py

KeyboardInterrupt: 

In [ ]:
ray.shutdown()

In [ ]:
gym.envs.registry.all()